Welcome to Preceptor Practice Session 10! 

In [17]:
import YData 


#YData.download_practice_code(10)              # Without Answers. 
#YData.download_practice_code(10, True)        # With Answers. 



import pandas as pd 
import numpy as np

#YData.download_data("loan_data.csv")


In [18]:
loans = pd.read_csv("loan_data.csv")
loans = loans.drop(columns = "loan_percent_income")
display(loans.head(3), loans.shape)

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,3.0,635,No,1


(45000, 13)

## 1. Linear regression

In regression, we try to predict a quantitative variable y, from a set of features X. 

Let's explore this by predicting the person's income (in USD) from other quantitative features in the above dataset. 

- You are free to choose your own set of features; I am following closely the setup from the class.
- Remark: When working with salaries, usually one takes `log transform`. You can use `np.log()` function for this. 


In [1]:
# get the features and the labels





- Let's use scikit-learn to generate training and test data (as we did previously for our KNN classifier). 

In [2]:
from sklearn.model_selection import train_test_split

# split data into a training and test set




- We can now create a new linear regression model, fit it to data, and make predictions. The method names are again very similar to what we used for the KNN classifier (i.e., the `fit()` and predict()` methods). 

In [3]:
from sklearn.linear_model import LinearRegression

# create a new linear regression modedl


In [4]:
# fit the model to our training data



In [5]:
# make predictions of the salaries on the test data



- We can assess the accuracy of our predictons using the root mean squared error which is defined as: 

$$RMSE = \sqrt{ \frac{1}{n}\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$$

- Here $\hat{y}$ is the predictions made by our linear model on the test data (i.e., the predicted salary) and y is the actual salary for the points in our test set.


In [7]:
# test the RMSE on the test data






- We can also use scikit-learn's `mean_squared_error()` to get the MSE, and we can use the `cross_val_score` to run k-fold cross-validation (again, in a very similar way to what we did for our KNN classifier). 

In [8]:
from sklearn.metrics import mean_squared_error

# Use scikit-learn's mean_squared_error() function to get the RMSE



In [11]:
# using cross-validation
from sklearn.model_selection import cross_val_score



- What does RMSE measures? 
- Answer: 


### Regression model equation

In linear regression, our predicted $\hat{y}$ values are given by the equation: $\hat{y} = b_0 + b_1 x_1 + ... + + b_k x_k$.

Let's fill out this equation for prediciting salary. 

To do this, let's start by extracting the intercept ($b_0$) and slope coefficients ($b_i's$) from our scikit-learn model.


In [12]:

# fit the linear regression model to our training data


# get the intercept and slope coefficients


# print out the coefficient values
 


- Given these coefficient values can you write our the regression equation for predicting person's salary? 
- Answer: 

#### Writing our own prediction function

Let's also write our own function called `get_predictions(b0_intercept, b_coefficients, X_data)` that takes the coefficient values and X values and returns predicted $\hat{y}$ values for each X value. In particular, the arguments to the function are:

1. `b0_intercept`: The linear regression intercept
2. `b_coefficients`: The linear regression slope coefficients
3. `X_data`: The X data values 

The returned value is a numpy ndarray of predictions for each X data point. 


In [13]:
# write a function to get the predictions


# get the predicted values on the test data



# see the it matches the scikit-learn predictions



## Inference on regression coefficients

We can also run inference procedures on our regression model using the statsmodel package. In particular, we can run hypothesis tests and create confidence intervals for our regression coefficents. 

When running a hypothesis test, our hypotheses are:

$H_0: \beta_i = 0$  
$H_A: \beta_i \ne 0$


In [14]:
# Hypothesis test on regression coeffients - which coefficients are statistically significantly different from zero? 
# (and confidence interval)

import statsmodels.api as sm

# add a constant value of 1 to our data


# fit the linear regression model using the OLS function


# get information on the regression coefficients found



- Would you drop `credit_score`? Why?
- Answer: 

# 2. Linear Regression with categorical features. 

In [19]:
# Let us recall our dataset. 

loans.tail(3)

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
44997,33.0,male,Associate,56942.0,7,RENT,2771.0,DEBTCONSOLIDATION,10.02,10.0,668,No,1
44998,29.0,male,Bachelor,33164.0,4,RENT,12000.0,EDUCATION,13.23,6.0,604,No,1
44999,24.0,male,High School,51609.0,1,RENT,6665.0,DEBTCONSOLIDATION,17.05,3.0,628,No,1


In [20]:
# Let us code Male = 0 and Female = 1. 





## Run Linear Regression with your choice of features but make sure to include `gender` as one of your features. 

In [21]:
# get the features and the labels





In [22]:
from sklearn.model_selection import train_test_split

# split data into a training and test set




In [23]:
from sklearn.linear_model import LinearRegression

# create a new linear regression modedl


In [24]:
# fit the model to our training data



In [25]:
# Hypothesis test on regression coeffients - which coefficients are statistically significantly different from zero? 
# (and confidence interval)

import statsmodels.api as sm

# add a constant value of 1 to our data


# fit the linear regression model using the OLS function


# get information on the regression coefficients found



-  Gender coefficient is $-1390.6578$. What does this mean?
-  Answer: 

- What is the p-value associated with Gender coefficient and how to interpret it?
- Answer: 

# 3. Open Ended: Build a predictive model to predict salaries using the dataset `loans` from above. 